In [30]:
import psycopg2 as ps
import warnings 
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import pandas.io.sql as sqlio
import datetime

## Подключение к базе данных и сбор датасетов

Подключение

In [31]:
conn = ps.connect(host="46.34.136.91", port = 25432, database="postgres", 
                  user="aaa_student", password="student_@@@_2022")

Сбор датасетов. User_info, Item_info выгружаем полностью

In [32]:
user_info = sqlio.read_sql_query("SELECT * FROM PUBLIC.services_verification_experiment_segment", conn)

In [5]:
user_info.head()

,user_id,experiment_group,UserType,RegistrationTime
0,117790500037,30_discount,Private,2015-05-12 11:28:29.494941
1,117869500025,60_discount,Private,2015-05-12 17:46:49.315601
2,118017750006,no_discount,Private,2015-05-13 05:16:35.901323
3,118060250024,60_discount,Private,2015-05-13 08:35:47.776235
4,118066250019,30_discount,Company,2015-05-13 08:57:25.346077


In [34]:
verification_info = sqlio.read_sql_query("SELECT * FROM PUBLIC.services_verification_discounts", conn)

In [35]:
verification_info.head()

,user_id,item_id,discount_day,experiment
0,49000737,11352751363,2021-10-01,1_verification_test
1,4255271,78048500109,2021-10-10,1_verification_test
2,111000153,101676750179,2021-10-02,1_verification_test
3,110750728,162344000369,2021-10-14,1_verification_test
4,143655000006,252307750608,2021-10-09,1_verification_test


Обращаем внимание, что скидку могут дать на несколько объявлений

In [10]:
verification_info.shape

(29963, 4)

In [11]:
verification_info['user_id'].unique().shape

(22658,)

In [12]:
verification_info.to_csv('verification_info.csv')

In [36]:
verification_info['discount_day'] = pd.to_datetime(verification_info['discount_day'], format='%Y-%m-%d')

Далее, сджойним таблицу с payments и таблицу с информацией по объявлениям. Ограничим таблицу item_vertical = 1, transaction_type = 3.

In [43]:
users_item_and_payment_info = sqlio.read_sql_query("SELECT * FROM PUBLIC.user_payments_info AS p LEFT JOIN PUBLIC.user_item_info As i ON p.user_id = i.user_id AND p.item_id = i.item_id WHERE item_vertical = '1' AND transaction_type = '3'", conn)

In [46]:
users_item_and_payment_info.to_csv('users_item_and_payment_info.csv')

Да, я, кстати, записал на всякий случай себе данные в csv таблицы) мало ли, потом соединение сломается)

In [44]:
users_item_and_payment_info.head()

,user_id,item_id,event_time,transaction_type,transaction_subtype,amount_net,item_id,user_id,item_vertical,item_region,item_creation_time
0,345629000019,1.074269e+12,2021-06-24 07:50:02,3,10,268.461538,1074269000371,345629000019,1,75,2021-06-22 21:44:28.775996
1,748996250017,1.087440e+12,2021-07-23 03:26:38,3,17,47.692308,1087439750349,748996250017,1,75,2021-07-13 07:11:55.396692
2,748996250017,1.087440e+12,2021-09-02 15:43:26,3,16,24.615385,1087439750349,748996250017,1,75,2021-07-13 07:11:55.396692
3,748996250017,1.087440e+12,2021-07-15 11:17:29,3,17,47.692308,1087439750349,748996250017,1,75,2021-07-13 07:11:55.396692
4,825285750064,1.087457e+12,2021-07-13 09:11:39,3,10,157.692308,1087456751087,825285750064,1,75,2021-07-13 09:11:42.829165


In [45]:
users_item_and_payment_info.shape

(906985, 11)

Небольшой прикол: доставая данные таким образом - получаются столбцы с одинаковыми именами) и возникают трудности с удалением дубликата (даже по номеру столбца). Пришлось написать функцию, которая переименовывает столбцы.

In [50]:
def rename_pandas_duplicated_columnes(df):
    cols = pd.Series(df.columns)
    for dup in cols[cols.duplicated()].unique(): 
        cols[cols[cols == dup].index.values.tolist()] = [dup + '.' + str(i) if i != 0 else dup for i in range(sum(cols == dup))]
    df.columns = cols
    return df

In [52]:
df = rename_pandas_duplicated_columnes(users_item_and_payment_info)

In [53]:
df.head()

,user_id,item_id,event_time,transaction_type,transaction_subtype,amount_net,item_id.1,user_id.1,item_vertical,item_region,item_creation_time
0,345629000019,1.074269e+12,2021-06-24 07:50:02,3,10,268.461538,1074269000371,345629000019,1,75,2021-06-22 21:44:28.775996
1,748996250017,1.087440e+12,2021-07-23 03:26:38,3,17,47.692308,1087439750349,748996250017,1,75,2021-07-13 07:11:55.396692
2,748996250017,1.087440e+12,2021-09-02 15:43:26,3,16,24.615385,1087439750349,748996250017,1,75,2021-07-13 07:11:55.396692
3,748996250017,1.087440e+12,2021-07-15 11:17:29,3,17,47.692308,1087439750349,748996250017,1,75,2021-07-13 07:11:55.396692
4,825285750064,1.087457e+12,2021-07-13 09:11:39,3,10,157.692308,1087456751087,825285750064,1,75,2021-07-13 09:11:42.829165


О, теперь разные )

In [54]:
df = df[df.columns[[0, 1, 2, 4, 5, 9, 10]]]

In [56]:
df.head()

,user_id,item_id,event_time,transaction_subtype,amount_net,item_region,item_creation_time
0,345629000019,1.074269e+12,2021-06-24 07:50:02,10,268.461538,75,2021-06-22 21:44:28.775996
1,748996250017,1.087440e+12,2021-07-23 03:26:38,17,47.692308,75,2021-07-13 07:11:55.396692
2,748996250017,1.087440e+12,2021-09-02 15:43:26,16,24.615385,75,2021-07-13 07:11:55.396692
3,748996250017,1.087440e+12,2021-07-15 11:17:29,17,47.692308,75,2021-07-13 07:11:55.396692
4,825285750064,1.087457e+12,2021-07-13 09:11:39,10,157.692308,75,2021-07-13 09:11:42.829165


Ну и, наверное, будет проще сразу сджойнить с информацией по юзерам и таблицей с верификацией.

In [57]:
df1 = df.merge(user_info, on='user_id', how='left')

In [58]:
df1.head()

,user_id,item_id,event_time,transaction_subtype,amount_net,item_region,item_creation_time,experiment_group,UserType,RegistrationTime
0,345629000019,1.074269e+12,2021-06-24 07:50:02,10,268.461538,75,2021-06-22 21:44:28.775996,60_discount,Shop,2017-10-13 23:19:36.356074
1,748996250017,1.087440e+12,2021-07-23 03:26:38,17,47.692308,75,2021-07-13 07:11:55.396692,60_discount,Private,2020-11-03 08:35:24.429217
2,748996250017,1.087440e+12,2021-09-02 15:43:26,16,24.615385,75,2021-07-13 07:11:55.396692,60_discount,Private,2020-11-03 08:35:24.429217
3,748996250017,1.087440e+12,2021-07-15 11:17:29,17,47.692308,75,2021-07-13 07:11:55.396692,60_discount,Private,2020-11-03 08:35:24.429217
4,825285750064,1.087457e+12,2021-07-13 09:11:39,10,157.692308,75,2021-07-13 09:11:42.829165,60_discount,Private,2021-03-17 13:52:10.158593


Сразу доберем в датасет информацию с верификациями, чтобы потом не возвращаться.

In [60]:
data = df1.merge(verification_info, on=['user_id', 'item_id'], how='left')

In [61]:
data.head()

,user_id,item_id,event_time,transaction_subtype,amount_net,item_region,item_creation_time,experiment_group,UserType,RegistrationTime,discount_day,experiment
0,345629000019,1.074269e+12,2021-06-24 07:50:02,10,268.461538,75,2021-06-22 21:44:28.775996,60_discount,Shop,2017-10-13 23:19:36.356074,NaT,NaN
1,748996250017,1.087440e+12,2021-07-23 03:26:38,17,47.692308,75,2021-07-13 07:11:55.396692,60_discount,Private,2020-11-03 08:35:24.429217,NaT,NaN
2,748996250017,1.087440e+12,2021-09-02 15:43:26,16,24.615385,75,2021-07-13 07:11:55.396692,60_discount,Private,2020-11-03 08:35:24.429217,NaT,NaN
3,748996250017,1.087440e+12,2021-07-15 11:17:29,17,47.692308,75,2021-07-13 07:11:55.396692,60_discount,Private,2020-11-03 08:35:24.429217,NaT,NaN
4,825285750064,1.087457e+12,2021-07-13 09:11:39,10,157.692308,75,2021-07-13 09:11:42.829165,60_discount,Private,2021-03-17 13:52:10.158593,NaT,NaN


Худо-бедно датасет собран. Идем считать метрики.

## Подсчет item_per_users, ARPU в группах до начала эксперимента

### item_per_users

Первым делом посчитаем item_per_user. Нас интересуют объявления, созданные до начала эксперимента: 2021-09-15, фильтровать будем item_creation_time этой датой.

In [68]:
df_for_item_per_users = data[['user_id', 'item_id', 'item_creation_time', 'experiment_group']]

In [69]:
df_for_item_per_users.head()

,user_id,item_id,item_creation_time,experiment_group
0,345629000019,1.074269e+12,2021-06-22 21:44:28.775996,60_discount
1,748996250017,1.087440e+12,2021-07-13 07:11:55.396692,60_discount
2,748996250017,1.087440e+12,2021-07-13 07:11:55.396692,60_discount
3,748996250017,1.087440e+12,2021-07-13 07:11:55.396692,60_discount
4,825285750064,1.087457e+12,2021-07-13 09:11:42.829165,60_discount


In [70]:
df_for_item_per_users['item_creation_time'] = pd.to_datetime(df_for_item_per_users['item_creation_time'])

In [71]:
df_for_item_per_users_before_test = df_for_item_per_users[df_for_item_per_users['item_creation_time'] < '2021-09-15']

In [76]:
item_counts = df_for_item_per_users_before_test.groupby(['experiment_group', 'user_id'])[['item_id']].count()

In [78]:
item_counts.head()

item_id
experiment_group user_id         
30_discount      789            1
                 2893           3
                 3449           2
                 3789           2
                 4137           5

Небольшой тест: действительно у юзера 2893 создано три объявления до 15 сентября

In [83]:
data.loc[(data['experiment_group'] == '30_discount') & (data['user_id'] == 2893)]

,user_id,item_id,event_time,transaction_subtype,amount_net,item_region,item_creation_time,experiment_group,UserType,RegistrationTime,discount_day,experiment
204672,2893,1.165118e+12,2021-11-24 09:56:45,10,101.538462,25,2021-11-24 09:56:48.784227,30_discount,Private,2010-12-24 08:48:26.573836,NaT,NaN
390935,2893,8.759838e+11,2021-08-24 06:45:39,10,101.538462,25,2020-08-25 10:37:17.954182,30_discount,Private,2010-12-24 08:48:26.573836,NaT,NaN
442461,2893,8.912750e+11,2021-09-06 13:17:12,10,50.846154,25,2020-09-20 23:32:12.640702,30_discount,Private,2010-12-24 08:48:26.573836,NaT,NaN
594267,2893,7.824090e+11,2021-08-01 12:47:52,10,50.846154,25,2020-01-21 22:44:31.264369,30_discount,Private,2010-12-24 08:48:26.573836,NaT,NaN


Найдем среднее количество items на одного юзера в каждой группе:

In [84]:
mean_item_per_user = item_counts.groupby('experiment_group').mean()

In [85]:
mean_item_per_user

,item_id
experiment_group,
30_discount,6.352235
60_discount,6.590310
no_discount,6.162525


### ARPU

Для подсчета ARPU до теста будем ограничивать event_time - время покупки

In [86]:
df_for_ARPU = data[['user_id', 'item_id', 'event_time', 'experiment_group', 'amount_net']]

In [88]:
df_for_ARPU.head()

,user_id,item_id,event_time,experiment_group,amount_net
0,345629000019,1.074269e+12,2021-06-24 07:50:02,60_discount,268.461538
1,748996250017,1.087440e+12,2021-07-23 03:26:38,60_discount,47.692308
2,748996250017,1.087440e+12,2021-09-02 15:43:26,60_discount,24.615385
3,748996250017,1.087440e+12,2021-07-15 11:17:29,60_discount,47.692308
4,825285750064,1.087457e+12,2021-07-13 09:11:39,60_discount,157.692308


Аналогично, ограничим время:

In [89]:
df_for_ARPU_before_test = df_for_ARPU[df_for_ARPU['event_time'] < '2021-09-15']

Посчитаем суммы платежей по юзерам:

In [92]:
user_payments = df_for_ARPU_before_test.groupby(['experiment_group', 'user_id'])[['amount_net']].sum()

In [93]:
user_payments.head()

amount_net
experiment_group user_id            
30_discount      789       37.692308
                 2893     203.230769
                 3449      69.307692
                 3789      60.153846
                 4137      40.461538

Для юзера 2893 в тесте выше сумма сходится )

Посчитаем ARPU

In [94]:
ARPU = user_payments.groupby('experiment_group').mean()

In [95]:
ARPU

,amount_net
experiment_group,
30_discount,230.898654
60_discount,239.768872
no_discount,226.872500
